In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import requests
from tqdm.auto import tqdm
from astropy.visualization import simple_norm
from astropy.coordinates import SkyCoord

from helpers import ObservationInfo, search_observations

# Search for observations

Using a sky coordinate, search for all matching observations with a certain radius.

In [ ]:
search_coords = SkyCoord('18h 08m 30s +08d00m00s')
search_coords = SkyCoord.from_name('M42')

search_coords

In [ ]:
search_results = search_observations(
    coords=search_coords,
    radius=10,
)

# Select observation from list

A `sequence_id` is used to explore observations. Here we randomly sample one observation to get the `sequence_id`.

In [ ]:
# Get the most recent (i.e. last) entry.
record = search_results.iloc[-10]
record.T

In [ ]:
sequence_id = record.sequence_id
sequence_id

In [ ]:
obs_info = ObservationInfo(sequence_id)

In [ ]:
cutout_size = (250, 250)

## Plot a single image

We can retrieve the data for the image by using the `get_image_data` function using the index of the image from the `raw_images` list.

An optional `coords` and `box_size` allow for cropping of the data as it is returned (the full data still travels over the network as of now).

You can also retrieve the raw or processed data using the `use_raw=True` flag.

In [ ]:
data0 = obs_info.get_image_data(idx=0, coords=search_coords, box_size=cutout_size, use_raw=True)

ax = plt.subplot(projection=data0.wcs)

norm = simple_norm(data0.data, 'sqrt', min_percent=5, max_percent=94)

im = ax.imshow(data0.data, origin='lower', norm=norm, cmap='Greys_r')
ax.figure.colorbar(im)
ax.figure.set_size_inches(12, 8)

plt.grid(color='white', ls='--', alpha=0.5);

We can perform this action in a loop to retrieve the same cutout for all the images in the observation.

Below we fetch the data for all the images into a list and then plot a single frame (similar to above), a median stack, and summation of all the data.

In [ ]:
stack_cutout = list()
for i, img in tqdm(enumerate(obs_info.raw_images), total=len(obs_info.raw_images), desc='Fetching image cutouts'):
    try:
        stack_cutout.append(obs_info.get_image_data(
            idx=i, 
            coords=search_coords, 
            box_size=cutout_size, 
            use_raw=True).data)
    except Exception:
        pass
                        
stack_cutout = np.array(stack_cutout)

In [ ]:
stack_cutout.shape

In [ ]:
diffs = pd.DataFrame(stack_cutout.reshape(stack_cutout.shape[0], -1).astype('float')).diff().values.reshape(stack_cutout.shape[0], *cutout_size)[1:]

In [ ]:
stretch = 'sqrt'

fig, axes = plt.subplots(ncols=2, nrows=2)

types = {
    'Single Frame': stack_cutout[0],
    'Difference': np.median(diffs, 0),
    'Median': np.median(stack_cutout, 0),
    'Sum': stack_cutout.sum(0),
}

for i, (title, data) in enumerate(types.items()):
    ax = axes.flatten()[i]
    im = ax.imshow(data, norm=simple_norm(data, stretch, min_percent=10, max_percent=96), origin='lower', cmap='Greys_r')
    fig.colorbar(im, ax=ax)
    ax.set_title(title)
    ax.axis('off')

fig.set_dpi(100)
fig.set_size_inches(13, 11)